In [77]:
import pandas as pd

In [78]:
total_costs = pd.ExcelFile('./data/data-consolidated.xlsx')

student_incomes = pd.ExcelFile('./data/mean-median-student-income.xlsx').parse('Columbia')
inflation_ratios = student_incomes[['year', 'inflation ratio']]

## Income stuff

In [79]:
# Median income in 2018 dollars for US and Columbia
us_income = total_costs.parse('US Trends').rename(
    columns={'Year': 'year', 'Median Income \n(in 2018 dollars)': 'income'}
)[[ 'year', 'income' ]]
columbia_income = student_incomes.rename(columns={'median - 2018 $': 'income'})[[ 'year', 'income' ]]

In [80]:
entity_incomes = [
    {'entity': 'United States', 'year': int(row['year']), 'income': int(row['income'])}
    for _, row in us_income.iterrows()
] + [
    {'entity': 'Columbia', 'year': int(row['year']), 'income': int(round(row['income']))}
    for _, row in columbia_income.iterrows()
]
entity_incomes

[{'entity': 'United States', 'year': 2018, 'income': 63572},
 {'entity': 'United States', 'year': 2017, 'income': 62846},
 {'entity': 'United States', 'year': 2016, 'income': 61894},
 {'entity': 'United States', 'year': 2015, 'income': 60236},
 {'entity': 'United States', 'year': 2014, 'income': 57377},
 {'entity': 'United States', 'year': 2013, 'income': 56492},
 {'entity': 'United States', 'year': 2012, 'income': 56077},
 {'entity': 'United States', 'year': 2011, 'income': 55769},
 {'entity': 'United States', 'year': 2010, 'income': 56748},
 {'entity': 'United States', 'year': 2009, 'income': 58115},
 {'entity': 'United States', 'year': 2008, 'income': 58611},
 {'entity': 'United States', 'year': 2007, 'income': 60664},
 {'entity': 'United States', 'year': 2006, 'income': 60205},
 {'entity': 'United States', 'year': 2005, 'income': 59679},
 {'entity': 'United States', 'year': 2004, 'income': 59182},
 {'entity': 'United States', 'year': 2003, 'income': 59655},
 {'entity': 'United Stat

## Get total costs for Ivies
**Right now unadjusted for inflation**

In [81]:
# The names of sheets in the excel file
ivies = ['Columbia', 'Brown', 'Cornell', 'Dartmouth', 'Harvard', 'Princeton', 'UPenn', 'Yale']

# Flatten the dataframes to: [ { school, year, cost } ]
school_costs = []
for ivy in ivies:
    sheet = total_costs.parse(ivy).rename(columns={'School Year (start)': 'year', 'School Year': 'year'})
    school_costs.append({
        'school': ivy,
        'data': sorted([
            {'year': int(row['year']), 'cost': int(row['total cost'])}
            for _, row in sheet[['year', 'total cost']].iterrows()
        ], key=lambda d: d['year'])
    })

school_costs = {'series': []}
for ivy in ivies:
    sheet = total_costs.parse(ivy).rename(columns={'School Year (start)': 'year', 'School Year': 'year'})
    school_costs['years'] = [int(row['year'])
            for _, row in sheet[['year', 'total cost']].iterrows()]
    school_costs['series'].append({
        'school': ivy,
        'values': [int(row['total cost'])
            for _, row in sheet[['year', 'total cost']].iterrows()],
    })
school_costs

{'series': [{'school': 'Columbia',
   'values': [76856,
    74199,
    71785,
    69084,
    66604,
    64144,
    61540,
    59208,
    56681,
    54294,
    51406,
    49260,
    46874,
    44646,
    42598,
    40140]},
  {'school': 'Brown',
   'values': [73802,
    71050,
    68106,
    65380,
    62694,
    60460,
    58140,
    56150,
    54370,
    52030,
    50560,
    48660,
    46340,
    44280,
    41538,
    39602]},
  {'school': 'Cornell',
   'values': [73904,
    70321,
    67613,
    65494,
    63606,
    61618,
    59591,
    57125,
    54676,
    52414,
    50384,
    48151,
    45877,
    43827,
    42049,
    40274]},
  {'school': 'Dartmouth',
   'values': [74359,
    71827,
    69474,
    67044,
    65133,
    63279,
    61398,
    58638,
    55386,
    52973,
    50547,
    48236,
    45963,
    43679,
    41851,
    39889]},
  {'school': 'Harvard',
   'values': [71650,
    69600,
    66900,
    64400,
    62250,
    59950,
    57050,
    56000,
    53950,
    5200

## Generate outputs

In [76]:
import json

# writing outputs…
with open('./outputs/school-costs.json', 'w') as f:
    json.dump(school_costs, f)
with open('./outputs/incomes.json', 'w') as f:
    json.dump(entity_incomes, f)

# …but also  directly writing to the recession finaid repo lol
with open('../recession-financial-aid/data/school-costs.json', 'w') as f:
    json.dump(school_costs, f, indent=2)